In [ ]:
# Pandas

import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['mpg', 'cyl', 'disp', 'hp', 'weight', 'acc', 'year', 'origin'] 

df = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
df

In [ ]:
df['mpg']

In [ ]:
df['mpg'].values[:5]

In [ ]:
df[['mpg', 'hp']]

In [ ]:
df[['mpg', 'hp']].values[:5]

In [ ]:
df.iloc[:5]

In [ ]:
cond = (df['year'] == 70)
df[cond]